In [22]:
function seprate_classes(x,y) # Separate the dataset into a subset of data for each class
    seprated_classes = Dict()
    for i in 1:length(x)
        feature = x[i]
        class_name = y[i]
        seprated_classes[class_name] = [] && class_name ∉ seprated_classes
        append!(seprated_classes[class_name],feature)
    end
    return seprate_classes
end

seprate_classes (generic function with 1 method)